## Solving monodomain equations on the slice with the Beeler-Reuter model

### Importing required libraries

In [1]:
%cd ..
from src.dynamics_models import *
from src.cell_models import *
from src.utils import *

/home/drvinko/Desktop/Modeliranje-biofizike-srcanog-tkiva


### Defining a domain, cell model and dynamics model and visualizing initial data

In [2]:
domain = heart_slice()

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Nurb)
Info    : [ 40%] Meshing curve 2 (Nurb)
Info    : [ 70%] Meshing curve 3 (Nurb)
Info    : Done meshing 1D (Wall 0.0291501s, CPU 0.02866s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0580124s, CPU 0.057896s)
Info    : Meshing 3D...
Info    : Done meshing 3D (Wall 2.2202e-05s, CPU 2.2e-05s)
Info    : 3628 nodes 7231 elements


In [3]:
class CellModel(BeelerReuter):
    def applied_current(self):
        def value(x):
            I_app_value = 10
            R1, R2, R3 = 0.2, 0.2, 0.1
            return I_app_value * (
                np.exp(-(((x[0] - 0.1) / R1) ** 2) - ((x[1] - 2.6) / R1) ** 2)
                + np.exp(-(((x[0] - 1.6) / R2) ** 2) - ((x[1] - 1.5) / R2) ** 2)
                + np.exp(-(((x[0] - 3.3) / R3) ** 2) - ((x[1] - 3.25) / R3) ** 2)
            )

        self.I_app.interpolate(value)
        self.I_app_duration = 20
        return self.I_app

In [4]:
class Model(MonodomainModel):
    def initial_V_m(self):
        self.V_m_n.x.array[:] = -85

    def conductivity(self):
        # Muscle fibres
        x_c, y_c = 0.2, 1
        self.fibers = ufl.as_vector(
            [
                -(self.x[1] - y_c)
                / ufl.sqrt((self.x[0] - x_c) ** 2 + (self.x[1] - y_c) ** 2),
                (self.x[0] - x_c)
                / ufl.sqrt((self.x[0] - x_c) ** 2 + (self.x[1] - y_c) ** 2),
            ]
        )

        # Healthy conductivities
        self.M_i = self.SIGMA_IT * ufl.Identity(2) + (
            self.SIGMA_IL - self.SIGMA_IT
        ) * ufl.outer(self.fibers, self.fibers)

In [5]:
cell_model = CellModel(domain)
model = Model(domain, cell_model)

In [6]:
plot_function(
    cell_model.I_app,
    camera_direction="xy",
    cmap="RdBu",
    clim = [0,2],
    save_to="monodomain_applied_current.pdf",
)

Widget(value='<iframe src="http://localhost:39633/index.html?ui=P_0x7f4335dd2990_0&reconnect=auto" class="pyvi…

### Solving equations with given parameters

In [7]:
model.solve(
    T=400,
    steps=10000,
    lambda_=0.8,
    signal_point=[4.3, 0.1, 0.0],
    save_to="monodomain.mp4",
    camera_direction="xy",
    checkpoints=[35, 50, 85, 100, 230, 300, 350, 400],
    checkpoint_file="monodomain",
)

Solving problem: 100%|██████████| 10000/10000 [21:26<00:00,  7.77it/s]


### Plotting fiber orientations

In [8]:
x_c, y_c = 0.2, 1
plot_vector_field(
    domain,
    lambda x: (
        -(x[1] - y_c) / np.sqrt((x[0] - x_c) ** 2 + (x[1] - y_c) ** 2),
        (x[0] - x_c) / np.sqrt((x[0] - x_c) ** 2 + (x[1] - y_c) ** 2),
        0,
    ),
    0.03,
    0.3,
    camera_direction='xy',
    save_to='monodomain_fibers.pdf'
)

Widget(value='<iframe src="http://localhost:39633/index.html?ui=P_0x7f437dd02510_1&reconnect=auto" class="pyvi…

### Saving a signal to a file

In [9]:
np.save('monodomain_signal.npy', model.signal)

ERROR:wslink.protocol:Exception raised
ERROR:wslink.protocol:KeyError('e0bf733efa75ca6a722b574586681766_13957d')
ERROR:wslink.protocol:Traceback (most recent call last):
  File "/home/drvinko/miniconda3/envs/diplomski/lib/python3.11/site-packages/wslink/protocol.py", line 332, in onMessage
    results = func(*args, **kwargs)
              ^^^^^^^^^^^^^^^^^^^^^
  File "/home/drvinko/miniconda3/envs/diplomski/lib/python3.11/site-packages/trame_vtk/modules/vtk/protocols/local_rendering.py", line 33, in get_array
    self.context.get_cached_data_array(data_hash, binary)
  File "/home/drvinko/miniconda3/envs/diplomski/lib/python3.11/site-packages/trame_vtk/modules/vtk/serializers/synchronization_context.py", line 30, in get_cached_data_array
    cache_obj = self.data_array_cache[p_md5]
                ~~~~~~~~~~~~~~~~~~~~~^^^^^^^
KeyError: 'e0bf733efa75ca6a722b574586681766_13957d'

